In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
file_path = '/Users/diana/Desktop/isear-test.xlsx'
custom_headers = ['Emotions', 'Text']
df = pd.read_excel(file_path, skiprows=1, header=None, names=custom_headers)

conda install openpyxl

In [3]:
df.head ()

,Emotions,Text
0,joy,My first feeling of falling in love. Coming a...
1,joy,When I saw that the mark I had obtained in an ...
2,joy,When I learnt that I had been admitted to the ...
3,anger,In a hurry to get to lectures and the car woul...
4,guilt,"Not attending school, when am not sick - but ..."


In [4]:
total_count = df['Emotions'].value_counts().sum()
print (total_count)

1150


In [5]:
df['Emotions'].value_counts()

Emotions
shame      182
fear       168
anger      166
disgust    166
sadness    161
guilt      155
joy        152
Name: count, dtype: int64

In [6]:
emotions_set = list (set (df['Emotions'].values))

In [7]:
#print (df['Text'].dtype)
df['Text'] = df['Text'].str.lower ()
#print (df['Text'])

text_column = df['Text']
tokenized_text = text_column.str.split()

for index, sentence in enumerate(tokenized_text):
    emotion_label = df['Emotions'][index]
    
    #print(emotion_label, sentence)

vocab = set() 
for tokens_list in tokenized_text:
    for token in tokens_list:
        if r'[^\w\s]' in token:
            if re.search(r'[^\w\s]', token):
                token = re.sub(r'[^\w\s]+$', '', token)
                if token not in vocab:
                    vocab.add (token[-1])
        if token not in vocab:
            vocab.add(token)
#print (list(vocab))


In [8]:
from math import log

collection = [token for instance in tokenized_text for token in instance]

#tf-idf

def calculate_tfidf(token, document):
    tf = document.count(token) / len(document)
    idf = log(1 + (len(tokenized_text) / collection.count(token)))
    tfidf = tf * idf
    return tfidf

tfidf_scores_list = []

for instance in tokenized_text:
        if token not in instance:
            tfidf_scores = {token: calculate_tfidf(token, instance) for token in set(instance)}
            #print (tfidf_scores)

In [9]:
#conda install scikit-learn

In [10]:
#import sklearn

In [11]:
"""from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the corpus to obtain the TF-IDF sparse matrix
tfidf_matrix_sparse = tfidf_vectorizer.fit_transform(sentence)

#tfidf_vector_first_sentence = tfidf_matrix_sparse[0]
#print("First", tfidf_vector_first_sentence)
print (tfidf_matrix_sparse)"""



'from sklearn.feature_extraction.text import TfidfVectorizer\n\n\ntfidf_vectorizer = TfidfVectorizer()\n\n# Fit and transform the corpus to obtain the TF-IDF sparse matrix\ntfidf_matrix_sparse = tfidf_vectorizer.fit_transform(sentence)\n\n#tfidf_vector_first_sentence = tfidf_matrix_sparse[0]\n#print("First", tfidf_vector_first_sentence)\nprint (tfidf_matrix_sparse)'

In [12]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from torch.autograd import Variable

In [13]:
class Emotions (nn.Module):

    def __init__(self, input_size, hidden_1, output_size):

        super (Emotions, self).__init__()
        self.fc1 = nn.Linear (input_size, hidden_1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear (hidden_1, output_size)
    def forward (self, x):
        out = self.fc1 (x)
        out = self.relu1 (out)
        out = self.fc2 (out)
        return out



In [14]:
model = Emotions (1150, 50, 7)

In [15]:
import torch.utils

batch_size = 50
train_loader = torch.utils.data.DataLoader (dataset = df, batch_size=batch_size, shuffle =True)